In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import numpy as np
import datetime
import time
import re

In [2]:
chromedriver_dir = r'C:\Users\LG\빅데이터 실습\chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(chromedriver_dir)

In [3]:
url = 'https://tv.naver.com/cjenm.STREETWOMANFIGHTER/playlists'

driver.get(url)
time.sleep(3)

In [4]:
# 플레이 리스트 다 보여주기 위함
# 더보기 계속 진행
while(1):
    close = 0
    try:
        xpath = """//*[@id="cds_flick"]/div/div[2]/div/div/div/div/div[3]/a"""
        driver.find_element_by_xpath(xpath).click()
    except:
        close = 1
        print("더보기 끝")
        
    if close == 1:
        break;

더보기 끝


In [5]:
# BeautifulSoup 실행

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup)

<html lang="ko"><head>
<title id="headTitle">
    	
    		네이버 TV
    		
    		
    	
    </title>
<meta charset="utf-8"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex" name="robots"/>
<meta content="summary" name="twitter:card"/>
<meta content="100012287952710" property="fb:admins"/>
<meta content="232491033870086" property="fb:app_id"/>
<meta content="https://tv.naver.com/cjenm.STREETWOMANFIGHTER" property="og:url"/>
<meta content="스트릿 우먼 파이터 : 네이버TV" property="og:title"/>
<meta content="" property="og:description"/>
<meta content="website" property="og:type"/>
<meta content="https://phinf.pstatic.net/tvcast/20210726_295/fiW8X_1627287664865lwyjo_JPEG/67ff5a4f-ad11-4662-a18a-b3bf8bf189ba.jpg?type=round_232_232" property="og:image"/>
<link href="/resources/release/css/tvcast_20211209161204.css" rel="stylesheet" type="text/css"/>
<

In [6]:
# 저장할 데이터 프레임
streetwomanfiger_view = pd.DataFrame(columns=("회차", "영상 이름", "조회수", "좋아요 수"))
streetwomanfiger_view

,회차,영상 이름,조회수,좋아요 수


In [7]:
# 해당 재생목록 가져오기 위한 재생 목록 이름 리스트
playlist_names = ['스트릿 우먼 파이터 9회', '스트릿 우먼 파이터 8회', '스트릿 우먼 파이터 7회',
                  '스트릿 우먼 파이터 6회', '스트릿 우먼 파이터 5회', '스트릿 우먼 파이터 4회',
                  '스트릿 우먼 파이터 3회', '스트릿 우먼 파이터 2회', '스트릿 우먼 파이터 1회']

In [8]:
playlists = soup.find_all('div', 'playlist _MM_CARD')
# 재생 목록 이름 리스트

for playlist in playlists:   
    playlist_name = playlist.find('h3', 'play_tit').get_text() # 재생 목록 이름 가져옴
    playlist_name = playlist_name[:13].strip() # 재생 목록을 프로그램 이름 + 회차까지 그리고 공백 제거
    
    playlist_name_list = [] # 재생 목록 이름 저장할 리스트
    vedio_name_list = [] # 영상 이름 저장할 리스트
    view_count = [] # 조회수 저장할 리스트
    like_count = [] # 좋아요 수 저장할 리스트
    
    if (playlist_name in playlist_names): # 회차에 해당하는 리스트만 가져옴
        vedio_names = playlist.find_all("a") # 영상 이름
        for vedio_name in vedio_names:
            vedio_name_tmp = vedio_name.select_one("dl dt a tooltip")
            if (vedio_name_tmp != None):
                vedio_name_tmp = vedio_name.select_one("dl dt a tooltip")['title'].strip()
                vedio_name_list.append(vedio_name_tmp)
    
        view_results = playlist.find_all('span', 'hit') # 조회수들 가져옴
        for view_result in view_results:
            view_result = view_result.get_text()
            view = int(view_result[4:].replace(',', '')) # 조회수 글자제외, int형 변환
            view_count.append(view) # 리스트에 조회수 저장
            playlist_name_list.append(playlist_name) # 리스트에 재생목록 이름 저장
            
        like_results = playlist.find_all('span', 'like') # 좋아요 수 가져옴
        for like_result in like_results:
            like_result = like_result.get_text().strip()
            like = int(like_result[5:].replace(',', '')) # 좋아요 수 글자제외, int형 변환
            like_count.append(like) # 리스트에 좋아요 수 저장
            
        # 데이터 프레임 생성
        df = pd.DataFrame([x for x in zip(playlist_name_list, vedio_name_list, view_count, like_count)],
                          columns=("회차", "영상 이름", "조회수", "좋아요 수"))
        # 앞에서 만들어 둔 데이터 프레임에 추가
        streetwomanfiger_view = streetwomanfiger_view.append(df)

In [9]:
streetwomanfiger_view

,회차,영상 이름,조회수,좋아요 수
0,스트릿 우먼 파이터 9회,[스우파] 파이널리스트 무대 모아보기 | 라치카 (LACHICA),44742,202
1,스트릿 우먼 파이터 9회,[스우파] 파이널리스트 무대 모아보기 | 코카N버터 (CocaNButter),12371,152
2,스트릿 우먼 파이터 9회,[스우파] 파이널리스트 무대 모아보기 | 홀리뱅 (HolyBang),28123,799
3,스트릿 우먼 파이터 9회,[스우파] 파이널리스트 무대 모아보기 | 훅 (HOOK),29165,582
4,스트릿 우먼 파이터 9회,[9회 예고] 반가운 노제&효진초이★ 오랜만에 모인 리더 8인?! 그리고 시작되는 ...,49639,287
...,...,...,...,...
45,스트릿 우먼 파이터 1회,[1회/리액션캠] Fight Judge 보아x태용x황상훈 | 프라우드먼 모니카 vs...,8616,165
46,스트릿 우먼 파이터 1회,[스우파/1회 하이라이트] '약자 지목 배틀' 텐션 폭발 무대,24157,225
47,스트릿 우먼 파이터 1회,[1회/풀캠] YGX 리정 vs 원트 이채연 @약자 지목 배틀 Full Cam,17440,129
48,스트릿 우먼 파이터 1회,[1회/리액션캠] MC 강다니엘 | YGX 리정 vs 원트 이채연 @약자 지목 배틀,11541,870


In [10]:
# csv 파일로 저장
# 2021-11-17 11:41 저장
streetwomanfiger_view.to_csv('streetwomanfiger_view_name.csv', index=False, encoding="utf-8-sig")

In [11]:
save_test = pd.read_csv('streetwomanfiger_view_name.csv')
save_test

,회차,영상 이름,조회수,좋아요 수
0,스트릿 우먼 파이터 9회,[스우파] 파이널리스트 무대 모아보기 | 라치카 (LACHICA),44742,202
1,스트릿 우먼 파이터 9회,[스우파] 파이널리스트 무대 모아보기 | 코카N버터 (CocaNButter),12371,152
2,스트릿 우먼 파이터 9회,[스우파] 파이널리스트 무대 모아보기 | 홀리뱅 (HolyBang),28123,799
3,스트릿 우먼 파이터 9회,[스우파] 파이널리스트 무대 모아보기 | 훅 (HOOK),29165,582
4,스트릿 우먼 파이터 9회,[9회 예고] 반가운 노제&효진초이★ 오랜만에 모인 리더 8인?! 그리고 시작되는 ...,49639,287
...,...,...,...,...
492,스트릿 우먼 파이터 1회,[1회/리액션캠] Fight Judge 보아x태용x황상훈 | 프라우드먼 모니카 vs...,8616,165
493,스트릿 우먼 파이터 1회,[스우파/1회 하이라이트] '약자 지목 배틀' 텐션 폭발 무대,24157,225
494,스트릿 우먼 파이터 1회,[1회/풀캠] YGX 리정 vs 원트 이채연 @약자 지목 배틀 Full Cam,17440,129
495,스트릿 우먼 파이터 1회,[1회/리액션캠] MC 강다니엘 | YGX 리정 vs 원트 이채연 @약자 지목 배틀,11541,870


In [12]:
save_test.groupby('회차').count()

,영상 이름,조회수,좋아요 수
회차,,,
스트릿 우먼 파이터 1회,50,50,50
스트릿 우먼 파이터 2회,87,87,87
스트릿 우먼 파이터 3회,42,42,42
스트릿 우먼 파이터 4회,35,35,35
스트릿 우먼 파이터 5회,33,33,33
스트릿 우먼 파이터 6회,52,52,52
스트릿 우먼 파이터 7회,64,64,64
스트릿 우먼 파이터 8회,71,71,71
스트릿 우먼 파이터 9회,63,63,63
